# Before you start
Open this link and create a shortcut to indices_genres in your drive: https://drive.google.com/file/d/1-0CjAdc5ZJIw_pxu8ycVBmHnJabAIxEg/view?usp=sharing

# Code

In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    colab = True
except:
    colab = False
print ("Running colab:", colab)
path = "/content/" if colab else ""
#abs path
import os
path = os.path.abspath(path) + "/"
path

Running colab: False


'/home/smoothjazzuser/Desktop/VQ-VAE-Search/'

In [2]:
!mkdir mel_specs_music
!mkdir mel_specs_music/train
!mkdir mel_specs_music/val
!mkdir mel_specs_music/train/cl
!mkdir mel_specs_music/val/cl
!git clone https://github.com/nadavbh12/VQ-VAE.git
!mv ./VQ-VAE/ ./VQ_VAE/ 

mkdir: cannot create directory ‘mel_specs_music’: File exists
mkdir: cannot create directory ‘mel_specs_music/train’: File exists
mkdir: cannot create directory ‘mel_specs_music/val’: File exists
mkdir: cannot create directory ‘mel_specs_music/train/cl’: File exists
mkdir: cannot create directory ‘mel_specs_music/val/cl’: File exists
fatal: destination path 'VQ-VAE' already exists and is not an empty directory.
mv: cannot move './VQ-VAE/' to './VQ_VAE/VQ-VAE': Directory not empty


In [3]:
#@title Hyperparams
import os
bs = 64
dict_size = 128
epochs = 15

h, w = 64, 256 #image size

vqvae_music_checkpoints_folder = path + 'vqvae_music_checkpoints/'
if not os.path.exists(vqvae_music_checkpoints_folder):
    os.mkdir(vqvae_music_checkpoints_folder)

ds_dir = path + 'mel_specs_music/val/'

In [4]:
import sklearn
from sklearn.model_selection import train_test_split
import glob
import pandas as pd
import pickle

import torch
from torchvision import datasets, transforms
from VQ_VAE.vq_vae.auto_encoder import VQ_CVAE
import tqdm
import numpy as np
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import plotly.express as px
import pandas as pd
import shutil

import librosa
import cv2
import scipy
import IPython.display as ipd

In [5]:
#@title Unzip, filter, undersample, and reorganize data
if colab: 
  !unzip '/content/drive/MyDrive/Big Data Project/large_music_mel_spectrograms.zip'

img = glob.glob(path + 'music_mel_spectrograms/*.png')

if colab:
  with open('/content/drive/MyDrive/indices_genres', 'rb') as f:
    ig = pickle.load(f)
else:
  with open('indices_genres', 'rb') as f:
    ig = pickle.load(f)

img_ids = list(map(lambda x: os.path.basename(x)[:-4], img))

df_filter = pd.DataFrame.from_dict({'genre': [ig[el] for el in img_ids], 'id':img}).dropna()
df_filter = pd.concat([df_filter[df_filter['genre'] == genre][:1000].reset_index(drop=True) for genre in list(df_filter['genre'].value_counts()[:-7].keys())]).reset_index(drop=True)
img = list(df_filter['id'].values)

img_train, img_test = train_test_split(img, test_size=0.2, random_state=42)
f_train = lambda x: path + 'mel_specs_music/train/cl/' + os.path.basename(x)
f_test = lambda x: path + 'mel_specs_music/val/cl/' + os.path.basename(x)
out_train = list(map(f_train, img_train))
out_test = list(map(f_test, img_test))

import shutil
for el1, el2 in zip(img_train, out_train):
  shutil.copy(el1, el2)

for el1, el2 in zip(img_test, out_test):
  shutil.copy(el1, el2)

In [6]:
#@title Tune augmentation hyperparameters
size = f'transforms.Resize(({h}, {w}))'
replace_main_with = '''dataset_transforms = {
    'custom': transforms.Compose([transforms.Grayscale(), transforms.Resize((h, w)), 
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5), (0.5))]),
    'imagenet': transforms.Compose([transforms.Grayscale(), transforms.Resize((h, w)), 
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5), (0.5))]),
    'cifar10': transforms.Compose([transforms.ToTensor(),
                                   transforms.Normalize((0.5), (0.5))]),
    'mnist': transforms.ToTensor()
}'''.replace('transforms.Resize((h, w))', size)
replace_main = '''dataset_transforms = {
    'custom': transforms.Compose([transforms.Resize(256), transforms.CenterCrop(256),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]),
    'imagenet': transforms.Compose([transforms.Resize(256), transforms.CenterCrop(256),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]),
    'cifar10': transforms.Compose([transforms.ToTensor(),
                                   transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]),
    'mnist': transforms.ToTensor()
}'''

with open(path + 'VQ_VAE/main.py', 'r') as f:
  data = f.read()

data = data.replace(replace_main, replace_main_with)

with open(path + 'VQ_VAE/main.py', 'w') as f:
  f.write(data)


replace_main_with = '''dataset_n_channels = {
    'custom': 1,
    'imagenet': 1,
    'cifar10': 1,
    'mnist': 1,
}'''
replace_main = '''dataset_n_channels = {
    'custom': 3,
    'imagenet': 3,
    'cifar10': 3,
    'mnist': 1,
}'''

with open(path + 'VQ_VAE/main.py', 'r') as f:
  data = f.read()

data = data.replace(replace_main, replace_main_with)

with open(path + 'VQ_VAE/main.py', 'w') as f:
  f.write(data)

In [7]:
#@title Audio Functions
def play_audio(audio_file, sr=22050):
    if type(audio_file) == str:
        return ipd.Audio(audio_file,  rate=sr)
    else:
        return ipd.Audio(audio_file, rate=sr)

def from_spectrogram(spectrogram,  hop_length=150, n_fft=2000, win=50, sr = 22050):
    # undo power_to_db
    S = spectrogram
    S = librosa.db_to_power(S)
    S = librosa.feature.inverse.mel_to_audio(S, sr=sr, n_fft=n_fft, hop_length=hop_length, window=win)
    return S

def png_to_audio(audio_file='spec.png', n_fft = 2000, hop_length = 150, win = 50, mi = -80.0, m = 0.0, sr = 22050, save=False):
    # read image
    spec = cv2.imread(audio_file, cv2.IMREAD_GRAYSCALE)

    # de_normalize
    spec = (spec * (m - mi) / 255) + mi
    spec = spec.astype(np.float32)

    # from spectrogram to audio
    aud = from_spectrogram(spec, n_fft=n_fft, hop_length=hop_length, win=win, sr=sr)

    # save audio
    aud = play_audio(aud)
    if save:
        with open(f'{audio_file[0:-4]}.wav', 'wb') as f:
                f.write(aud.data)

    return aud

In [8]:
#@title Train and save to drive (be sure to save the previous trained model somewhere else as the checkpoint folder will be emptied)
VQVAE_path = path + 'VQ_VAE'
%cd $VQVAE_path
if colab:
    !python3 main.py --dataset=custom --model=vqvae --data-dir=/content/mel_specs_music --epochs={epochs} --batch-size {bs} --dict-size {dict_size}
    checkpoint_path = sorted(glob.glob('/content/VQ_VAE/results/*/checkpoints/*.pth'))[-1]
else:
    !python3 main.py --dataset=custom --model=vqvae --data-dir=../mel_specs_music --epochs={epochs} --batch-size {bs} --dict-size {dict_size}
    checkpoint_path = sorted(glob.glob('../VQ_VAE/results/*/checkpoints/*.pth'))[-1]

!rm -rf {vqvae_music_checkpoints_folder}
!mkdir {vqvae_music_checkpoints_folder}

shutil.copyfile(checkpoint_path, os.path.join(vqvae_music_checkpoints_folder, os.path.basename(checkpoint_path))) 

/home/smoothjazzuser/Desktop/VQ-VAE-Search/VQ_VAE
Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
NumExpr defaulting to 8 threads.
Train Epoch: 1 [    0/7232 ( 0%)]   time: 1.64   mse_train: 0.080826 vq_train: 0.001638 commitment_train: 0.001638
Train Epoch: 1 [  640/7232 ( 8%)]   time: 0.70   mse_train: 0.445213 vq_train: 0.014806 commitment_train: 0.014806
Train Epoch: 1 [ 1280/7232 (17%)]   time: 0.70   mse_train: 0.288412 vq_train: 0.013540 commitment_train: 0.013540
Train Epoch: 1 [ 1920/7232 (26%)]   time: 0.69   mse_train: 0.218011 vq_train: 0.013013 commitment_train: 0.013013
Train Epoch: 1 [ 2560/7232 (35%)]   time: 0.70   mse_train: 0.196570 vq_train: 0.013176 commitment_train: 0.013176
Train Epoch: 1 [ 3200/7232 (44%)]   time: 0.70   mse_train: 0.159391 vq_train: 0.013314 commitment_train: 0.013314
Train Epoch: 1 [ 3840/7232 (53%)]   time: 0.75   mse_train: 0.119247 vq_train: 0.013376 commitment_train: 0.013376
Train Epoch: 1 

'/home/smoothjazzuser/Desktop/VQ-VAE-Search/vqvae_music_checkpoints/model_9.pth'

In [9]:
#@title Load model
model = VQ_CVAE(128, k=dict_size, num_channels=1) # k has to be equal to dict_size!!!
model.load_state_dict(torch.load(checkpoint_path))

<All keys matched successfully>

In [10]:
#@title Predict values
if not colab:
  !cd ~./VQ-VAE-SEARCH
T = transforms.Compose([transforms.Grayscale(), transforms.Resize((h, w)), #transforms.CenterCrop(256),
                        transforms.ToTensor(),
                        transforms.Normalize((0.5), (0.5))])

test_dataset = torch.utils.data.DataLoader(datasets.ImageFolder(ds_dir, transform=T), batch_size=bs, shuffle=False)

categories = []
data_names = datasets.ImageFolder(ds_dir, transform=T)
data_names = [el[0] for el in data_names.samples]
test_ids = list(map(lambda x: os.path.basename(x)[:-4], data_names))
categories = [ig[el] for el in test_ids]

normalize = lambda x: (x - x.min()) / (x.max() - x.min())

all_outputs = []
i = 0
for data, _ in tqdm.tqdm(test_dataset):
  with torch.no_grad():
    outputs = model(data)[2]
    outputs = outputs.reshape(outputs.shape[0], -1).detach().cpu().numpy() #outputs[1] = enc, outputs[2] = emb
  all_outputs.append(outputs)
  i += 1

all_outputs = np.concatenate(all_outputs, 0)
#all_outputs = normalize(all_outputs)

with open(f'{vqvae_music_checkpoints_folder}/all_outputs.pickle', 'wb') as f:
  pickle.dump(all_outputs, f)

/bin/bash: line 1: cd: ~./VQ-VAE-SEARCH: No such file or directory


100%|██████████| 29/29 [01:12<00:00,  2.49s/it]


In [11]:
#@title Top K data
n_comp = 50
with open(f'{vqvae_music_checkpoints_folder}/all_outputs.pickle', 'rb') as f:
  all_outputs = pickle.load(f)

categories = []
data_names = datasets.ImageFolder(ds_dir, transform=T)
data_names = [el[0] for el in data_names.samples]
test_ids = list(map(lambda x: os.path.basename(x)[:-4], data_names))
categories = [ig[el] for el in test_ids]

spec_paths = datasets.ImageFolder(ds_dir, transform=T)
spec_paths = [el[0] for el in spec_paths.imgs]

pca = PCA(n_components=n_comp)
pca_result = pca.fit_transform(all_outputs)
pca_df = pd.DataFrame(pca_result, columns=list(map(str,list(range(0, n_comp)))))
pca_df['categories'] = categories
pca_df['spec_paths'] = spec_paths

In [ ]:
#@title Top K
colab = True
#search = audio_search(sr=22050)
select_id = 599
k = 10
vecs = pca_df[list(map(str,list(range(0, n_comp))))].to_numpy()
distances = sklearn.metrics.pairwise.cosine_similarity(vecs, vecs)
top_k = np.flip(np.argsort(distances[select_id]))[:k]
top_k_df = pca_df.iloc[top_k][['categories', 'spec_paths']]
audios = [png_to_audio(spec) for spec in top_k_df['spec_paths'].values]
display(top_k_df)

In [ ]:
for audio in audios:
  display(audio)

In [ ]:
#@title PCA
pca = PCA(n_components=3)
pca_result = pca.fit_transform(all_outputs)
pca_df = pd.DataFrame(pca_result, columns=['1', '2', '3'])
pca_df['categories'] = categories

fig = px.scatter_3d(pca_df, x='1', y='2', z='3', color='categories', width=1500)
fig.show()

In [ ]:
#@title TSNE (requires tuning)

pca_tsne = PCA(n_components=50) # change the number of components as a hyperparameter
pca_tsne_result = pca_tsne.fit_transform(all_outputs)

tsne = TSNE(n_components=3, verbose=1, perplexity=25, n_iter=3000, learning_rate=200)
tsne_results = tsne.fit_transform(pca_tsne_result)
tsne_df = pd.DataFrame(tsne_results, columns=['1', '2', '3'])
tsne_df['categories'] = categories

fig = px.scatter_3d(tsne_df, x='1', y='2', z='3', color='categories', width=1200)
fig.show()